## Imports

In [ ]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
from getpass import getpass
import sys, os
import pandas as pd
from datetime import date, datetime, timedelta
import os
import pandas as pd
from datetime import date, datetime


## Set influx connection

In [ ]:
### get influx credentials from file (not in repo., see template)
thisDir=os.getcwd()
if os.path.isdir(thisDir):
    print("directory found:",thisDir)
    sys.path.insert(1, thisDir)
    import connectionDetails
    connectionDict=connectionDetails.GetInfluxCredentials()
    #print(credDict)
    print("done.")
else:
    print("no directory found:",thisDir)

In [ ]:
### set credentials
bucket_remote = connectionDict['bucket']
org_remote = connectionDict['org']
token_remote = connectionDict['token']
url_remote = connectionDict['url']

print("Connection details:")
print(f"bucket_remote: {bucket_remote}")
print(f"org_remote: {org_remote}")
print(f"token_remote: {token_remote}")
print(f"url_remote: {url_remote}")


## Query Data

The code below takes in the flux data as well as the data from the picolog folder to perform calculations on them and produce an output in the csv file 'output_file'

In [ ]:
### geting temp., humid. data from influx
def fetch_temperature_and_humidity(start_time_str, bucket_remote, url_remote, token_remote, org_remote):
    clientV2_remote = influxdb_client.InfluxDBClient(url=url_remote, token=token_remote, org=org_remote)
    query_api_remote = clientV2_remote.query_api()

    start_time = datetime.strptime(str(start_time_str), '%Y-%m-%d_%H:%M:%S')

    filters = {'_measurement': "data"}
    query_str = f'from(bucket: \"{bucket_remote}\") |> range(start: {start_time.strftime("%Y-%m-%dT%H:%M:%SZ")})'
    for k, v in filters.items():
        query_str += f' |> filter(fn: (r) => r["{k}"] == "{v}")'
    query_str += ' |> yield(name: "mean")'

    query_result_remote = query_api_remote.query_data_frame(org=org_remote, query=query_str)

    temperature_results = []
    humidity_results = []
    for index, row in query_result_remote.iterrows():
        if row['_field'] == 'temperature':
            temperature_results.append(row.to_dict())
        elif row['_field'] == 'humidity':
            humidity_results.append(row.to_dict())

    if len(temperature_results) > 0:
        temperature = temperature_results[0]['_value']
    else:
        temperature = None

    if len(humidity_results) > 0:
        humidity = humidity_results[0]['_value']
    else:
        humidity = None

    return temperature, humidity


In [ ]:
### derive values from picoLogger raw data
def perform_calculations(input_folder, output_file, bucket_remote, url_remote, token_remote, org_remote, timestamp_str):
    all_data = []
    
    # Convert the timestamp to the desired format
    timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d_%H:%M:%S").strftime("%Y-%m-%dT%H:%M:%SZ")
    
    for filename in os.listdir(input_folder):
        if filename.endswith('.csv'):
            file_path = os.path.join(input_folder, filename)
            data = pd.read_csv(file_path)
            
            # Calculate Vin Drop and GND Drop
            data['Vin Drop (V)'] = data['Vin+ Last (V)'] - data['Vin- Last (V)']
            data['GND Drop (V)'] = data['GND+ Last (V)'] - data['GND- Last (V)']
            
            # Calculate Resistance Vin(Ohms) and Total Resistance(mOhms)
            data['Resistance Vin(Ohms)'] = data['Vin Drop (V)'] / 10 / 5
            data['Resistance GND(Ohms)'] = data['GND Drop (V)'] / 10 / 5
            data['Total Resistance(mOhms)'] = (data['Resistance Vin(Ohms)'] + data['Resistance GND(Ohms)']) * 1000
            
            # Calculate Capacitor Equivalent leakage current(nA)
            data['Capacitor Equivalent leakage current(nA)'] = ((data['Capacitor leakage test Last (V)'] / 10) / (1 * 10 ** 6)) / 1000000
            
            # Calculate NTC value(Kohms)
            data['NTC value (Kohms)'] = 0.2 * 51 / data['NTC Last (V)']
            
            # Fetch temperature and humidity
            temperature, humidity = fetch_temperature_and_humidity(timestamp_str, bucket_remote, url_remote, token_remote, org_remote)
            
            # Prepare the new DataFrame with desired columns and values
            result_data = pd.DataFrame({
                'component': filename,
                'componentType': 'PCB',
                'stage': 'PCB_QC',
                'testType': 'HV_LV_TEST',
                'institution': 'Glasgow',
                'runNumber': data.shape[0],
                'date': date.today().strftime('%Y-%m-%d'),
                'passed': 'YES',
                'problems': 'False',
                'property1_value': 'B.masic',
                'property1_key': 'OPERATOR',
                'property2_value': 'INSTRUMENT',
                'property2_key': '',
                'property3_value': 'ANALYSIS_VERSION',
                'property3_key': '',
                'result1_key': 'VIN_DROP',
                'result1_value': data['Vin Drop (V)'],
                'result2_key': 'GND_DROP',
                'result2_value': data['GND Drop (V)'],
                'result3_key': 'EFFECTIVE RESISTANCE',
                'result3_value': data['Total Resistance(mOhms)'],
                'result4_key': 'HV_LEAKAGE',
                'result4_value': data['Capacitor leakage test Last (V)'],
                'result5_key': 'LEAKAGE_CURRENT (nA)',
                'result5_value': data['Capacitor Equivalent leakage current(nA)'],
                'result6_key': 'NTC_VOLTAGE',
                'result6_value': data['NTC Last (V)'],
                'result7_key': 'NTC_VALUE',
                'result7_value': data['NTC value (Kohms)'],
                'result8_key': 'TERMPERATURE',
                'result8_value': temperature,
                'result9_key': 'HUMIDITY',
                'result9_value': humidity,
                'timestamp': [timestamp]  # Add the timestamp column to the DataFrame
            })
            
            all_data.append(result_data)
    
    if all_data:
        final_result = pd.concat(all_data, ignore_index=True)
        final_result.to_csv(output_file, index=False)
        print(f"Calculation completed. Output saved to {output_file}.")


In [ ]:
### run conversion
input_folder = thisDir+"/../picolog_folder"
output_file = thisDir+"/../output_file.csv"
timestamp_str = "2023-08-01_23:00:00"  # Replace this with your desired timestamp

perform_calculations(input_folder, output_file, bucket_remote, url_remote, token_remote, org_remote, timestamp_str)
